In [1]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.autograd import Variable
import torch
from tqdm import tqdm
import sys

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
sys.path.insert(0,'..')
import bitmap
import itertools
from forward_prediction import forward_model

# Generate data

## Training data

In [4]:
regenerate_data = False

In [5]:
# Generate or load training data
N = 51200

if regenerate_data:
    train_data_gen = bitmap.generate_train_set(N, 41, min_delta=1, max_delta=1)
    deltas, start_boards, stop_boards = map(np.array, zip(*list(train_data_gen)))
    # Save training data
    np.save('../../data/training_start_boards', start_boards)
    np.save('../../data/training_stop_boards', stop_boards)
else:
    start_boards = np.load('../../data/training_start_boards.npy')
    stop_boards = np.load('../../data/training_stop_boards.npy')

## Validation data

In [55]:
# Generate or load validation data
N_valid = 1280

if regenerate_data:
    valid_data_gen = bitmap.generate_train_set(N_valid, 1024, min_delta=1, max_delta=1)
    deltas, valid_start_boards, valid_stop_boards = map(np.array, zip(*list(valid_data_gen)))
    # Save validation data
    np.save('../../data/valid_start_boards', valid_start_boards)
    np.save('../../data/valid_stop_boards', valid_stop_boards)
else:
    valid_start_boards = np.load('../../data/valid_start_boards.npy')
    valid_stop_boards = np.load('../../data/valid_stop_boards.npy')

In [56]:
X_valid = Variable(torch.tensor(valid_start_boards).view(N_valid, 1, 25, 25).float())
y_valid = Variable(torch.tensor(valid_stop_boards).view(N_valid, 1, 25, 25).float())

## Test data

In [8]:
# Generate or load test data
N_test = 25600

if regenerate_data:
    test_data_gen = bitmap.generate_train_set(N, 42, min_delta=1, max_delta=1)
    deltas, test_start_boards, test_stop_boards = map(np.array, zip(*list(test_data_gen)))
    # Save test data
    np.save('../../data/test_start_boards', test_start_boards)
    np.save('../../data/test_stop_boards', test_stop_boards)
else:
    test_start_boards = np.load('../../data/test_start_boards.npy')
    test_stop_boards = np.load('../../data/test_stop_boards.npy')

# Model trainer

In [68]:
def train(model, X, y, X_valid, y_valid, 
          optim, criterion, output_path, num_epochs=50, batch_size=128):
    # Release CUDA memory
    torch.cuda.empty_cache()

    # Set optimizer
    optimizer = optim(model.parameters())
    
    # Setup Tensorboard (https://pytorch.org/docs/stable/tensorboard.html)
    writer = SummaryWriter()
    writer.add_graph(model.cpu(), X)
    model.cuda()

    # Train
    n_iter = 0
    for epoch in range(num_epochs): 
        permutation = torch.randperm(X.size()[0])
        running_loss = 0.0
        pbar = tqdm(range(0, X.size()[0], batch_size))
        for i in pbar:
            n_iter += 1
            indices = permutation[i:i+batch_size]
            batch = X[indices].cuda()
            target = y[indices].cuda()
        
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            pbar.set_description("[{:d}, {:5d}] loss: {:.8f}".format(epoch + 1, i + 1, loss.item()))    
            
            # Calculate MAE
            output_boards = (outputs > 0.5).int()
            mae = torch.sum(output_boards != target) / (batch_size * 25 * 25)
            
            # Write data to Tensorboard
            writer.add_scalar('Loss/train', loss.item(), n_iter)
            writer.add_scalar('MAE/train', mae.item(), n_iter)
            
            # Write boards and validation results to Tensorboard every 50 batches
            if n_iter % 50 == 0:
                with torch.no_grad():
                    model.eval()
                    valid_loss = 0
                    valid_mae = 0
                    m = 0
                    for j in range(0, X_valid.size()[0], batch_size):
                        m += 1
                        valid_batch = X_valid[j:j+batch_size].cuda()
                        valid_target = y_valid[j:j+batch_size].cuda()
                        valid_outputs = model(valid_batch)
                        valid_loss += criterion(valid_outputs, valid_target)
                        valid_boards = (valid_outputs > 0.5).int()
                        valid_mae += torch.sum(valid_boards != valid_target).float()
                    valid_loss /= m
                    valid_mae /= (X_valid.size()[0] * 25 * 25)
                    writer.add_image('predicted stop board', valid_boards[-1], n_iter)
                    writer.add_image('actual stop board', y_valid[-1], n_iter)
                    writer.add_scalar('Loss/valid', valid_loss.item(), n_iter)
                    writer.add_scalar('MAE/valid', valid_mae.item(), n_iter)
    writer.close()
    # Save model state_dict
    torch.save(model.state_dict(), output_path)

# Train a forward network

In [37]:
class ForwardNet(nn.Module):
    def __init__(self):
        super(ForwardNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 8, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')
        
    def forward(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [38]:
forward_net = ForwardNet()
criterion = nn.BCELoss()

In [39]:
X = Variable(torch.tensor(start_boards[:25600]).view(25600, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(stop_boards[:25600]).view(25600, 1, 25, 25).float())

In [40]:
vanilla_forward_model_path = "../models/johnson/vanilla_forward.pkl"
# train(forward_net, X, y, X_valid, y_valid, optim.Adam, criterion, vanilla_forward_model_path, num_epochs=30)

In [41]:
def get_forward_mae(model, weight_path, start_boards, stop_boards, n):
    # Load model
    model.load_state_dict(torch.load(weight_path))
    # Convert boards to tensor
    start_boards_tensor = torch.tensor(start_boards[:n]).view(n, 1, 25, 25).float()
    stop_boards_tensor = torch.tensor(stop_boards[:n]).view(n, 1, 25, 25)
    with torch.no_grad():
        model.eval()
        # Make prediction
        predicted_stop_board = (model(start_boards_tensor) > 0.5).int()
        error = torch.sum(predicted_stop_board != stop_boards_tensor)
        # print(predicted_stop_board)
        # print(stop_boards_tensor)
        return error / (n * 25 * 25)

In [42]:
# Training data MAE
train_mae = get_forward_mae(forward_net, vanilla_forward_model_path, 
                            start_boards, stop_boards, 25600)
print("The training data MAE is {:.8f}.".format(train_mae))

# Test data MAE
test_mae = get_forward_mae(forward_net, vanilla_forward_model_path,
                           test_start_boards, test_stop_boards, N_test)
print("The test data MAE is {:.8f}.".format(test_mae))

The training data MAE is 0.00000000.
The test data MAE is 0.00000000.


# Relax starting boards

In [43]:
# Modify starting boards 
def relax_boards(boards):
    np.random.seed(41)
    return np.abs(np.random.rand(*boards.shape) / 2 - boards)

In [44]:
relaxed_start_boards = relax_boards(start_boards)

In [57]:
class RelaxedForwardNet(nn.Module):
    def __init__(self):
        super(RelaxedForwardNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv0 = nn.Conv2d(1, 8, (1, 1))
        self.activ0 = nn.ReLU()
        self.conv1 = nn.Conv2d(8, 16, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ1 = nn.PReLU()
        self.conv2 = nn.Conv2d(16, 8, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')
        
    def forward(self, x):
        x = self.activ0(self.conv0(x))
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [58]:
relaxed_forward_net = RelaxedForwardNet()
criterion = nn.BCELoss()

In [59]:
X_relaxed = Variable(torch.tensor(relaxed_start_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float())

In [60]:
relaxed_valid_start_boards = relax_boards(valid_start_boards)
X_valid_relaxed = Variable(torch.tensor(relaxed_valid_start_boards).view(N_valid, 1, 25, 25).float())

In [ ]:
relaxed_forward_model_path = "../models/johnson/relaxed_forward.pkl"
train(relaxed_forward_net, X_relaxed, y, X_valid_relaxed, y_valid, optim.Adam, criterion, relaxed_forward_model_path, num_epochs=10)

[10,  2177] loss: 0.01652307:   4%|▍         | 18/400 [00:01<00:21, 17.77it/s]

In [ ]:
# Training data MAE
train_mae = get_forward_mae(relaxed_start_boards, stop_boards, N)
print("The training data MAE is {:.6f}.".format(train_mae))

# Test data MAE
relaxed_test_start_boards = relax_boards(test_start_boards)
test_mae = get_forward_mae(relaxed_test_start_boards, test_stop_boards, N_test)
print("The test data MAE is {:.6f}.".format(test_mae))

# Reverse model

In [ ]:
class ReverseNet(nn.Module):
    def __init__(self):
        super(ReverseNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ1 = nn.PReLU()
        self.conv2 = nn.Conv2d(16, 8, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')

    def forward(self, x):
        x = self.reverse(x)
        return x

    def reverse(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [ ]:
net = ReverseNet()
print(net)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters()) #, lr=0.001, momentum=0.9)

In [ ]:
X = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(start_boards).view(N, 1, 25, 25).float())
num_epochs = 50
batch_size = 128
for epoch in tqdm(range(num_epochs)): 
    permutation = torch.randperm(X.size()[0])
    running_loss = 0.0
    for i in range(0, X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch = X[indices]
        target = y[indices]
        
        optimizer.zero_grad()
        outputs = net(batch)
        # loss = criterion(outputs, batch)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 0 and i > 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

In [ ]:
# Generate testing data
N_test = 25600
# test_data_gen = bitmap.generate_test_set(N_test, 45, max_delta=1)
test_data_gen = bitmap.generate_test_set(N_test, 45, min_delta=5, max_delta=5)
delta, test_stop_boards = map(list, zip(*list(test_data_gen)))

In [ ]:
def get_mae(stop_boards, n, delta=1):
    stop_boards_tensor = torch.tensor(stop_boards).view(n, 1, 25, 25).float()
    with torch.no_grad():
        net.eval()
        # Make prediction
        output = (net(stop_boards_tensor) > 0.5).int()
        # Evolve one step forward
        predicted_stop_board = output
        for _ in range(delta):
            predicted_stop_board = (forward_model.forward(predicted_stop_board) > 0.5).int()
        error = torch.sum(predicted_stop_board != stop_boards_tensor.int())
        return error / (n * 25 * 25)

In [ ]:
# Training data MAE
train_mae = get_mae(stop_boards, N, 5).item()
print("The training data MAE is {:.6f}.".format(train_mae))

# Test data MAE
test_mae = get_mae(test_stop_boards, N_test, 5).item()
print("The test data MAE is {:.6f}.".format(test_mae))

In [ ]:
# Test data MAE
test_mae = get_mae(test_stop_boards, N_test, 5).item()
print("The test data MAE is {:.6f}.".format(test_mae))

# Appendix: CUDA memory management

In [ ]:
torch.cuda.memory_summary()

In [ ]:
torch.cuda.memory_snapshot()